In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
#from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score , confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
import pickle
import joblib

In [33]:
df_ = pd.read_pickle("../data/processed/final_processed_data.pkl")

In [34]:
df = df_.copy()

In [35]:
# db = df.head(1000) # for test 

In [36]:
# db.to_csv('../data/raw/Testdb2.csv', index=False)

In [37]:
df.drop(df.tail(100).index,inplace=True)
df

,Name,Parent_full_DNA_Seq,Child_full_DNA_Seq,ParentM,ParentF,target
0,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0
1,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0
2,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGACTCCCGTG...,NaN,NaN,1
3,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,1
4,A1,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,0
...,...,...,...,...,...,...
14895,E14895,CTCCGTCGACGCTTTAGGGACATAGATAGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0
14896,E14896,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0
14897,E14897,CTCCGTTGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0
14898,E14898,CTCCGTTGACGCTTTAAGGACATAAATGGGAGCGCTGATTCCCGTG...,NaN,NaN,NaN,0


In [38]:
df['Parent_full_DNA_Seq'] = df['Parent_full_DNA_Seq'].astype('str')
df['Child_full_DNA_Seq'] = df['Child_full_DNA_Seq'].astype('str')

In [39]:
df_pre = df[['Parent_full_DNA_Seq','Child_full_DNA_Seq','target']]

In [40]:
df_pre

,Parent_full_DNA_Seq,Child_full_DNA_Seq,target
0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,0
1,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,0
2,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGACTCCCGTG...,1
3,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,1
4,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,0
...,...,...,...
14895,CTCCGTCGACGCTTTAGGGACATAGATAGGAGCTCTGATTCCCGTG...,nan,0
14896,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,nan,0
14897,CTCCGTTGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,nan,0
14898,CTCCGTTGACGCTTTAAGGACATAAATGGGAGCGCTGATTCCCGTG...,nan,0


In [41]:
x = df_pre.drop('target',axis=1)
y = df_pre['target']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [42]:
x.shape , y.shape

((58484, 2), (58484,))

In [43]:
x

,Parent_full_DNA_Seq,Child_full_DNA_Seq
0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...
1,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...
2,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGACTCCCGTG...
3,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...
4,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...
...,...,...
14895,CTCCGTCGACGCTTTAGGGACATAGATAGGAGCTCTGATTCCCGTG...,nan
14896,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,nan
14897,CTCCGTTGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,nan
14898,CTCCGTTGACGCTTTAAGGACATAAATGGGAGCGCTGATTCCCGTG...,nan


In [44]:
y

0        0
1        0
2        1
3        1
4        0
        ..
14895    0
14896    0
14897    0
14898    0
14899    0
Name: target, Length: 58484, dtype: int64

In [45]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [46]:
kmer_embeddings_c = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/train_vectorizer_np/kmer_embeddings_c1.npy')
kmer_embeddings_p = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/train_vectorizer_np/kmer_embeddings_p1.npy')

In [47]:
# # vectorizer = CountVectorizer(analyzer = 'char' ,ngram_range=(3,3) , min_df=1)
# vectorizer = joblib.load('../data/interim/kmer_model.sav')
# X_ch = vectorizer.fit_transform(X_train['Child_full_DNA_Seq'])
# X_p = vectorizer.fit_transform(X_train['Parent_full_DNA_Seq'])
# kmer_embeddings_c = X_ch.toarray()
# kmer_embeddings_p = X_p.toarray()

In [48]:
kmer_embeddings_p = np.pad(kmer_embeddings_p, ((0, 0), (0, 1)), 'constant')
for i in range(len(kmer_embeddings_c[0])):
    column_name_child = f'child_gene_k_{i}'
    column_name_parent = f'parent_gene_k_{i}'
    X_train[column_name_child] = [kmer_embeddings_c[j][i] for j in range(len(X_train))]
    X_train[column_name_parent] = [kmer_embeddings_p[j][i] for j in range(len(X_train))]

# Drop specific ['child_gene_k_64', 'parent_gene_k_64'] columns if present in train data 
columns_to_drop = ['child_gene_k_64', 'parent_gene_k_64']
X_train.drop(columns_to_drop, axis=1, errors='ignore', inplace=True)

/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_41972/2377178368.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[column_name_parent] = [kmer_embeddings_p[j][i] for j in range(len(X_train))]
/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_41972/2377178368.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train[column_name_child] = [kmer_embeddings_c[j][i] for j in range(len(X_train))]


In [49]:
X_train = X_train.drop(['Parent_full_DNA_Seq','Child_full_DNA_Seq'],axis=1)

In [50]:
kmer_embeddings_child_test = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/test_vectorizer_np/kmer_embeddings_child_test1.npy')
kmer_embeddings_parent_test = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/test_vectorizer_np/kmer_embeddings_parent_test1.npy')

In [51]:
# # Working with test data
# X_t_c= vectorizer.transform(X_test['Child_full_DNA_Seq'])
# # X_t_c vect
# X_t_p = vectorizer.transform(X_test['Parent_full_DNA_Seq'])
# # X_t_p >>>>>>>> vectorizer_0
# kmer_embeddings_child_test = X_t_c.toarray()
# kmer_embeddings_parent_test = X_t_p.toarray()

In [52]:
for i in range (0, len(kmer_embeddings_child_test[0])):
    X_test['child_gene_k_'+str(i)] = [kmer_embeddings_child_test[j][i] for j in range (0, len(X_test))]
    X_test['parent_gene_k_'+str(i)] = [kmer_embeddings_parent_test[j][i] for j in range (0, len(X_test))]

/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_41972/477141918.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test['parent_gene_k_'+str(i)] = [kmer_embeddings_parent_test[j][i] for j in range (0, len(X_test))]
/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_41972/477141918.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test['child_gene_k_'+str(i)] = [kmer_embeddings_child_test[j][i] for j in range (0, len(X_test))]


In [53]:
X_test.shape , X_train.shape

((17546, 130), (40938, 128))

In [54]:
X_test.drop(['Parent_full_DNA_Seq','Child_full_DNA_Seq'],axis=1,inplace=True)

In [55]:
X_test

,child_gene_k_0,parent_gene_k_0,child_gene_k_1,parent_gene_k_1,child_gene_k_2,parent_gene_k_2,child_gene_k_3,parent_gene_k_3,child_gene_k_4,parent_gene_k_4,...,child_gene_k_59,parent_gene_k_59,child_gene_k_60,parent_gene_k_60,child_gene_k_61,parent_gene_k_61,child_gene_k_62,parent_gene_k_62,child_gene_k_63,parent_gene_k_63
4684,0,1231,0,1227,0,1188,0,1173,0,1140,...,0,1195,0,1111,0,1179,0,1212,0,1166
11948,1018,1072,1127,1169,1139,1160,962,983,1110,1145,...,1155,1131,995,1021,1249,1242,1137,1142,1082,1070
3521,0,1155,0,1198,0,1183,0,1103,0,1125,...,0,1211,0,1169,0,1195,0,1166,0,1159
14376,0,1124,0,1209,0,1179,0,1125,0,1163,...,0,1135,0,1132,0,1151,0,1155,0,1141
19896,1064,1086,1151,1151,1180,1183,1019,1012,1088,1107,...,1203,1192,1059,1046,1220,1261,1111,1113,1086,1133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11544,1104,1102,1179,1156,1105,1109,1046,1059,1162,1109,...,1151,1145,1092,1088,1179,1184,1145,1143,1119,1108
6920,983,1059,1132,1103,1174,1175,1035,1036,1150,1153,...,1174,1192,1032,1045,1214,1218,1180,1172,1098,1074
12086,1089,1052,1167,1121,1161,1161,1016,1018,1079,1125,...,1188,1189,1020,1025,1176,1171,1177,1132,1167,1129
90,1016,1037,1129,1154,1212,1154,1047,1041,1040,1067,...,1202,1195,1052,1062,1229,1212,1155,1146,1093,1050


In [56]:
X_train

,child_gene_k_0,parent_gene_k_0,child_gene_k_1,parent_gene_k_1,child_gene_k_2,parent_gene_k_2,child_gene_k_3,parent_gene_k_3,child_gene_k_4,parent_gene_k_4,...,child_gene_k_59,parent_gene_k_59,child_gene_k_60,parent_gene_k_60,child_gene_k_61,parent_gene_k_61,child_gene_k_62,parent_gene_k_62,child_gene_k_63,parent_gene_k_63
10137,0,1115,0,1132,0,1111,0,1129,0,1134,...,0,1159,0,1110,0,1221,0,1177,0,1132
12390,0,1126,0,1304,0,1177,0,1103,0,1175,...,0,1185,0,1128,0,1212,0,1212,0,1082
1743,0,1159,0,1193,0,1166,0,1128,0,1176,...,0,1179,0,1136,0,1232,0,1164,0,1065
8717,0,1103,0,1203,0,1205,0,1144,0,1163,...,0,1198,0,1118,0,1204,0,1161,0,1132
14068,1061,1061,1169,1189,1154,1149,1012,1023,1184,1153,...,1255,1171,1155,1059,1059,1201,1196,1146,1154,1116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10759,0,1127,0,1205,0,1152,0,1160,0,1205,...,0,1147,0,1172,0,1159,0,1169,0,1115
16466,1054,1029,1154,1160,1173,1187,1077,1043,1141,1176,...,1238,1160,1152,1090,1077,1175,1180,1168,1145,1082
860,1001,1044,1130,1151,1187,1145,1019,1040,1137,1104,...,1230,1202,1178,1088,1070,1184,1202,1142,1126,1152
15895,1037,1036,1147,1135,1149,1124,1055,1055,1128,1122,...,1259,1097,1107,1078,1062,1208,1209,1096,1096,1091


In [57]:
# X_train = X_train.drop(['child_gene_k_64','parent_gene_k_64'],axis=1)

In [58]:
model_xgb = xgb.XGBClassifier(
    learning_rate=0.3,
    n_estimators=15000,
    max_depth=15,
)


In [59]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# transform data
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [60]:
#train_x = X_train.values
#test_x = X_test.values

In [61]:
# grid_search = GridSearchCV(estimator=ada_boost, param_grid=param_grid, scoring='accuracy', cv=5)
# grid_search.fit(X_train, y_train)

In [62]:
model_xgb.fit(X_train, y_train)
# 9 min 51 sec  test/ 1
# 30 min 22 sec  test/ 2
# 8 min 23 sec  test/ 3

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=15000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [63]:
y_pred_xgb = model_xgb.predict(X_test)

In [64]:
y_pred_xgb

array([0, 1, 0, ..., 0, 0, 0])

In [65]:
accuracy = accuracy_score(y_test, y_pred_xgb)
precision = precision_score(y_test, y_pred_xgb)
recall = recall_score(y_test, y_pred_xgb)
f1 = f1_score(y_test, y_pred_xgb)
conf_matrix = confusion_matrix(y_test, y_pred_xgb)

In [66]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("\n")

Accuracy: 80.51%
Precision: 75.46%
Recall: 70.62%
F1 Score: 72.96%
Confusion Matrix:
[[9514 1500]
 [1919 4613]]




In [ ]:
# with open('model_baseline.pkl' , 'wb') as f:
#   pickle.dump(model_xgb, f)

## Validation of the model & Testing New Data

- i will extract the new data from the csv file
- i will use the model to predict the labels for the new data

In [67]:
df_tt = pd.read_csv('../data/raw/Testdb2.csv')

In [ ]:
df_tt

In [68]:
df_tt.target.value_counts()

1    515
0    485
Name: target, dtype: int64

In [69]:
kmer_embeddings_child_test = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/test_vectorizer_np/kmer_embeddings_child_test1.npy')
kmer_embeddings_parent_test = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/test_vectorizer_np/kmer_embeddings_parent_test1.npy')

In [70]:
for i in range (0, len(kmer_embeddings_child_test[0])):
    df_tt['child_gene_k_'+str(i)] = [kmer_embeddings_child_test[j][i] for j in range (0, len(df_tt))]
    df_tt['parent_gene_k_'+str(i)] = [kmer_embeddings_parent_test[j][i] for j in range (0, len(df_tt))]

/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_41972/2247410066.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tt['parent_gene_k_'+str(i)] = [kmer_embeddings_parent_test[j][i] for j in range (0, len(df_tt))]
/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_41972/2247410066.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tt['child_gene_k_'+str(i)] = [kmer_embeddings_child_test[j][i] for j in range (0, len(df_tt))]


In [71]:
y_dd = df_tt['target']

In [72]:
df_tt.columns

Index(['Name', 'Parent_full_DNA_Seq', 'Child_full_DNA_Seq', 'ParentM',
       'ParentF', 'target', 'child_gene_k_0', 'parent_gene_k_0',
       'child_gene_k_1', 'parent_gene_k_1',
       ...
       'child_gene_k_59', 'parent_gene_k_59', 'child_gene_k_60',
       'parent_gene_k_60', 'child_gene_k_61', 'parent_gene_k_61',
       'child_gene_k_62', 'parent_gene_k_62', 'child_gene_k_63',
       'parent_gene_k_63'],
      dtype='object', length=134)

In [73]:
df_tt.drop(['Name','Parent_full_DNA_Seq','Child_full_DNA_Seq','ParentM','ParentF', 'target'],axis=1,inplace=True)

In [ ]:
df_tt

In [74]:
y_pred_ = model_xgb.predict(df_tt)

In [ ]:
y_pred_

In [75]:
accuracy = accuracy_score(y_dd, y_pred_)
precision = precision_score(y_dd, y_pred_)
recall = recall_score(y_dd, y_pred_)
f1 = f1_score(y_dd, y_pred_)
conf_matrix = confusion_matrix(y_dd, y_pred_)

In [76]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("\n")

Accuracy: 46.00%
Precision: 48.32%
Recall: 69.90%
F1 Score: 57.14%
Confusion Matrix:
[[100 385]
 [155 360]]




# Model Pipeline:

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import FunctionTransformer
# import pandas as pd
# import joblib

# # Function to create DataFrame
# def create_data_frame(child_seq, possible_parent_seq):
#     repeated_value = child_seq[0]
#     data = {
#         'possiable_child': [repeated_value] * len(possible_parent_seq),
#         'possiable_parent': possible_parent_seq
#     }
#     # Convert the dictionary to a pandas DataFrame
#     test_df = pd.DataFrame(data)
#     return test_df

# # Function to convert columns to categorical
# def to_categorical(df_):
#     df_ca = df_.copy()
#     df_ca['possiable_child'] = df_ca['possiable_child'].astype('category')
#     df_ca['possiable_parent'] = df_ca['possiable_parent'].astype('category')
#     return df_ca

# # Function to encode DataFrame
# def encode_df(df_):
#     df_c = df_.copy()
#     # Perform encoding using loaded_model or any other appropriate method
#     # Example: loaded_model.transform(df_c['possiable_child'])
#     # Example: loaded_model.transform(df_c['possiable_parent'])
#     # For demonstration purposes, let's assume loaded_model is defined elsewhere
#     # and applied as a TfidfVectorizer
#     loaded_model = joblib.load('../data/interim/kmer_model.sav')
#     X_t_c = loaded_model.fit_transform(df_c['possiable_child'])
#     X_t_p = loaded_model.fit_transform(df_c['possiable_parent'])
    
#     kmer_embeddings_child_test = X_t_c.toarray()
#     kmer_embeddings_parent_test = X_t_p.toarray()
#     for i in range (0, len(kmer_embeddings_child_test[0])):
#         df_c['child_gene_k_'+str(i)] = [kmer_embeddings_child_test[j][i] for j in range (0, len(df_c))]
#         df_c['parent_gene_k_'+str(i)] = [kmer_embeddings_parent_test[j][i] for j in range (0, len(df_c))]
        
#     return df_c.drop(['possiable_child' , 'possiable_parent'] , axis = 1)

# # Define the pipeline steps
# preprocessing_pipeline = ColumnTransformer([
#     ('encode_df', FunctionTransformer(encode_df), ['possiable_child', 'possiable_parent'])
# ])

# # Create the pipeline
# pipeline = Pipeline([
#     ('create_df', FunctionTransformer(create_data_frame, kw_args={'possible_parent_seq': possible_parent_seq})),
#     ('to_categorical', FunctionTransformer(to_categorical)),
#     ('preprocessing', preprocessing_pipeline)
# ])

# # Example usage:
# child_seq = 'ACTG'
# possible_parent_seq = ['ACTG', 'CGTA', 'GATC']
# processed_df = pipeline.transform(create_data_frame(child_seq, possible_parent_seq))
# print(processed_df.head())


In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import FunctionTransformer
# import pandas as pd
# import joblib

# def create_data_frame(child_seq, possible_parent_seq):
#     repeated_value = child_seq[0]
#     data = {
#         'possiable_child': [repeated_value] * len(possible_parent_seq),
#         'possiable_parent': possible_parent_seq
#     }
#     # Convert the dictionary to a pandas DataFrame
#     test_df = pd.DataFrame(data)
#     return test_df


# # Function to convert columns to categorical
# def to_categorical(df_):
#     df_ca = df_.copy()
#     df_ca['possiable_child'] = df_ca['possiable_child'].astype('category')
#     df_ca['possiable_parent'] = df_ca['possiable_parent'].astype('category')
#     return df_ca

# # Function to encode DataFrame
# def encode_df(df_):
#     df_c = df_.copy()
#     loaded_model = joblib.load('../data/interim/kmer_model.sav')
#     X_t_c = loaded_model.fit_transform(df_c['possiable_child'])
#     X_t_p = loaded_model.fit_transform(df_c['possiable_parent'])
    
#     kmer_embeddings_child_test = X_t_c.toarray()
#     kmer_embeddings_parent_test = X_t_p.toarray()
#     for i in range (0, len(kmer_embeddings_child_test[0])):
#         df_c['child_gene_k_'+str(i)] = [kmer_embeddings_child_test[j][i] for j in range (0, len(df_c))]
#         df_c['parent_gene_k_'+str(i)] = [kmer_embeddings_parent_test[j][i] for j in range (0, len(df_c))]
        
#     return df_c.drop(['possiable_child' , 'possiable_parent'] , axis = 1)

# # Define the pipeline steps
# preprocessing_pipeline = ColumnTransformer([
#     ('encode_df', FunctionTransformer(encode_df), ['possiable_child', 'possiable_parent'])
# ])

# # Create the pipeline
# pipeline = Pipeline([
#     ('create_df', FunctionTransformer(create_data_frame, kw_args={'possible_parent_seq': possible_parent_seq})),
#     ('to_categorical', FunctionTransformer(to_categorical)),
#     ('preprocessing', preprocessing_pipeline)
# ])

# processed_df = pipeline.transform(create_data_frame(child_seq, possible_parent_seq))
# print(processed_df.head())


In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import FunctionTransformer
# from sklearn.feature_extraction.text import CountVectorizer
# from xgboost import XGBClassifier
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Load data
# df_pre = pd.read_pickle("../data/processed/final_processed_data.pkl")
# x = df_pre.drop('target', axis=1)
# y = df_pre['target']
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# df_tt = pd.read_csv('../data/raw/Testdb2.csv')
# y_dd = df_tt['target']



# vectorizer = CountVectorizer(analyzer = 'char' ,ngram_range=(3,3) , min_df=1)
# X_ch = vectorizer.fit_transform(X_train['Child_full_DNA_Seq'])
# X_p = vectorizer.fit_transform(X_train['Parent_full_DNA_Seq'])
# kmer_embeddings_child = X_ch.toarray()
# kmer_embeddings_parent = X_p.toarray()

# # Define a function to extract kmer embeddings
# def extract_kmer_embeddings(X, kmer_embeddings_child, kmer_embeddings_parent):
#     X_copy = pd.DataFrame(X)  # Convert to DataFrame
#     for i in range(len(kmer_embeddings_child[0])):
#         X_copy['child_gene_k_' + str(i)] = [kmer_embeddings_child[j][i] for j in range(len(X_copy))]
#         X_copy['parent_gene_k_' + str(i)] = [kmer_embeddings_parent[j][i] for j in range(len(X_copy))]
#     return X_copy.drop(['Parent_full_DNA_Seq', 'Child_full_DNA_Seq'], axis=1)

# # Define the pipeline steps
# preprocessing_pipeline = Pipeline([
#     ('kmer_embeddings', FunctionTransformer(func=extract_kmer_embeddings, kw_args={'kmer_embeddings_child': kmer_embeddings_child, 'kmer_embeddings_parent': kmer_embeddings_parent})),
#     ('scale_numeric', ColumnTransformer([
#         ('scale', StandardScaler(), ['child_gene_k_' + str(i) for i in range(65)] + ['parent_gene_k_' + str(i) for i in range(65)])
#     ]))
# ])

# model_pipeline = Pipeline([
#     ('preprocessing', preprocessing_pipeline),
#     ('classifier', XGBClassifier(learning_rate=0.3, n_estimators=15000, max_depth=15))
# ])

# # Fit the pipeline
# model_pipeline.fit(X_train, y_train)

# # Predictions
# y_pred_test = model_pipeline.predict(X_test)
# y_pred_validation = model_pipeline.predict(df_tt)

# # Evaluate the model
# accuracy_test = accuracy_score(y_test, y_pred_test)
# accuracy_validation = accuracy_score(y_dd, y_pred_validation)

# print("Test Accuracy:", accuracy_test)
# print("Validation Accuracy:", accuracy_validation)


In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import FunctionTransformer
# from sklearn.feature_extraction.text import CountVectorizer
# from xgboost import XGBClassifier
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Load data
# df_pre = pd.read_pickle("../data/processed/final_processed_data.pkl")
# x = df_pre.drop('target', axis=1)
# y = df_pre['target']
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# df_tt = pd.read_csv('../data/raw/Testdb2.csv')
# y_dd = df_tt['target']

# # Function to convert DNA sequences to strings
# def convert_to_string(df):
#     df_copy = df.copy()
#     df_copy['Child_full_DNA_Seq'] = df_copy['Child_full_DNA_Seq'].astype(str)
#     df_copy['Parent_full_DNA_Seq'] = df_copy['Parent_full_DNA_Seq'].astype(str)
#     return df_copy

# # Define a function to extract kmer embeddings
# def extract_kmer_embeddings(X, kmer_embeddings_child, kmer_embeddings_parent):
#     X_copy = pd.DataFrame(X)  # Convert to DataFrame
#     num_samples = len(X_copy)
#     num_features_child = len(kmer_embeddings_child[0])
#     num_features_parent = len(kmer_embeddings_parent[0])
    
#     # Check if the dimensions match
#     if num_samples != len(kmer_embeddings_child) or num_samples != len(kmer_embeddings_parent):
#         raise ValueError("Dimensions of input data and embeddings do not match.")
    
#     for i in range(num_features_child):
#         if i >= num_features_child or i >= num_features_parent:
#             raise IndexError("Index out of bounds.")
#         X_copy['child_gene_k_' + str(i)] = [kmer_embeddings_child[j][i] for j in range(num_samples)]
#         X_copy['parent_gene_k_' + str(i)] = [kmer_embeddings_parent[j][i] for j in range(num_samples)]
#     return X_copy.drop(['Parent_full_DNA_Seq', 'Child_full_DNA_Seq'], axis=1)


# # Define the pipeline steps
# preprocessing_pipeline = Pipeline([
#     ('convert_to_string', FunctionTransformer(func=convert_to_string)),
#     ('vectorize', ColumnTransformer([
#         ('count_vectorizer_child', CountVectorizer(analyzer='char', ngram_range=(3, 3), min_df=1), 'Child_full_DNA_Seq'),
#         ('count_vectorizer_parent', CountVectorizer(analyzer='char', ngram_range=(3, 3), min_df=1), 'Parent_full_DNA_Seq')
#     ])),
#     ('kmer_embeddings', FunctionTransformer(func=extract_kmer_embeddings, kw_args={'kmer_embeddings_child': kmer_embeddings_child, 'kmer_embeddings_parent': kmer_embeddings_parent})),
#     ('scale_numeric', ColumnTransformer([
#         ('scale', StandardScaler(), ['child_gene_k_' + str(i) for i in range(64)] + ['parent_gene_k_' + str(i) for i in range(64)])
#     ]))
# ])

# model_pipeline = Pipeline([
#     ('preprocessing', preprocessing_pipeline),
#     ('classifier', XGBClassifier(learning_rate=0.3, n_estimators=15000, max_depth=15))
# ])

# # Fit the pipeline
# model_pipeline.fit(X_train, y_train)

# # Predictions
# y_pred_test = model_pipeline.predict(X_test)
# y_pred_validation = model_pipeline.predict(df_tt)

# # Evaluate the model
# accuracy_test = accuracy_score(y_test, y_pred_test)
# accuracy_validation = accuracy_score(y_dd, y_pred_validation)

# print("Test Accuracy:", accuracy_test)
# print("Validation Accuracy:", accuracy_validation)


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import FunctionTransformer
# from xgboost import XGBClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Load data
# df_pre = pd.read_pickle("../data/processed/final_processed_data.pkl")
# x = df_pre.drop('target', axis=1)
# y = df_pre['target']
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# df_tt = pd.read_csv('../data/raw/Testdb2.csv')
# y_dd = df_tt['target']

# # Example embeddings arrays
# kmer_embeddings_child = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/test_vectorizer_np/kmer_embeddings_child_test.npy')
# kmer_embeddings_parent = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/test_vectorizer_np/kmer_embeddings_parent_test.npy')
# kmer_embeddings_child = np.pad(kmer_embeddings_child, ((0, 0), (0, 1)), 'constant')
# kmer_embeddings_parent = np.pad(kmer_embeddings_parent, ((0, 0), (0, 1)), 'constant')

# def extract_kmer_embeddings(X, kmer_embeddings_child, kmer_embeddings_parent):
#     X_copy = pd.DataFrame(X)  # Convert to DataFrame
#     num_samples = len(X_copy)
#     num_features_child = len(kmer_embeddings_child[0])
#     num_features_parent = len(kmer_embeddings_parent[0])
    
#     # Check if the dimensions match
#     if num_samples != len(kmer_embeddings_child) or num_samples != len(kmer_embeddings_parent):
#         raise ValueError("Dimensions of input data and embeddings do not match.")
    
#     for i in range(num_features_child):
#         if i >= num_features_child or i >= num_features_parent:
#             raise IndexError("Index out of bounds.")
#         X_copy['child_gene_k_' + str(i)] = [kmer_embeddings_child[j][i] for j in range(num_samples)]
#         X_copy['parent_gene_k_' + str(i)] = [kmer_embeddings_parent[j][i] for j in range(num_samples)]
    
#     return X_copy.drop(['Parent_full_DNA_Seq', 'Child_full_DNA_Seq'], axis=1), None, None

# # Define the pipeline steps
# preprocessing_pipeline = ColumnTransformer([
#     ('kmer_embeddings', FunctionTransformer(func=extract_kmer_embeddings, kw_args={'kmer_embeddings_child': kmer_embeddings_child, 'kmer_embeddings_parent': kmer_embeddings_parent}), ['Parent_full_DNA_Seq', 'Child_full_DNA_Seq']),
#     ('scale_numeric', StandardScaler(), ['child_gene_k_' + str(i) for i in range(64)] + ['parent_gene_k_' + str(i) for i in range(64)])
# ])

# model_pipeline = Pipeline([
#     ('preprocessing', preprocessing_pipeline),
#     ('classifier', XGBClassifier(learning_rate=0.3, n_estimators=15000, max_depth=15))
# ])

# # Fit the pipeline
# model_pipeline.fit(X_train, y_train)

# # Predictions
# y_pred_test = model_pipeline.predict(X_test)
# y_pred_validation = model_pipeline.predict(df_tt)

# # Evaluate the model
# accuracy_test = accuracy_score(y_test, y_pred_test)
# accuracy_validation = accuracy_score(y_dd, y_pred_validation)

# print("Test Accuracy:", accuracy_test)
# print("Validation Accuracy:", accuracy_validation)


In [ ]:
# print(f"Accuracy: {accuracy * 100:.2f}%")
# print(f"Precision: {precision * 100:.2f}%")
# print(f"Recall: {recall * 100:.2f}%")
# print(f"F1 Score: {f1 * 100:.2f}%")
# print("Confusion Matrix:")
# print(conf_matrix)
# print("\n")

# ------------------
# 1- model_xgb.pkl
# Accuracy: 51.00%
# Precision: 50.78%
# Recall: 43.96%
# F1 Score: 47.12%
# Confusion Matrix:
# [[175 127]
#  [167 131]]
# ------------------
# 2- best_ada_boost_model.pkl
# Accuracy: 49.00%
# Precision: 49.23%
# Recall: 85.57%
# F1 Score: 62.50%
# Confusion Matrix:
# [[ 39 263]
#  [ 43 255]]
# ------------------
# 3- model_xgb_clf.pkl
# ValueError: y must have at least two dimensions for multi-output regression but has only one.
# ------------------
# 4- model_xgb_clf_hyperparameter.pkl
# ValueError: y must have at least two dimensions for multi-output regression but has only one.


In [ ]:
# # saving Model
# with open('model_xgb.pkl' , 'wb') as f:
#   pickle.dump(model_xgb, f)